In [1]:

from pyspark.sql import SparkSession
import pyspark

spark = SparkSession.builder \
    .appName("first_app") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/danieldoh/.ivy2/cache
The jars for the packages stored in: /Users/danieldoh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-93a02aa9-f606-4979-be91-6cb7650a2861;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central

In [2]:
# Creating the dataframe and subscribing it to the health events topic. We will likely have to create multiple dataframes for the 6 topics


df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "44.201.154.178:9092") \
  .option("subscribe", "health_events") \
  .option("startingOffsets", "latest") \
  .load()



['key', 'value', 'topic', 'partition', 'offset', 'timestamp', 'timestampType']

In [17]:

# A lot of the regular methods you would use on a spark dataframe, because we're using streaming data.
#That's why I'm trying to capture a screenshot of the data and save it to a file. 
#if you can do it in a different way, power to you.

from pyspark.sql.functions import *
from pyspark.sql.types import *
import time 


query = df.selectExpr("CAST(value as STRING)", "topic", "partition", "timestamp")\
    .outputMode("append") \
    .format("csv") \
    .option("path", "file:///") \
    .option("checkpointLocation", "file:///") \
    .start()
timeout_seconds = 20
start_time = time.time()
while (time.time() - start_time) < timeout_seconds and query.isActive:
    time.sleep(1)

if query.isActive == True:
    query.stop()
# # Wait for the query to finish
# query.awaitTermination()
